# 규진님이 주신 데이터셋 1

In [241]:
# http://boostcamp.stages.ai/competitions/4/discussion/post/174
import pandas as pd
train = pd.read_csv("../input/data/train/train.tsv",sep='\t',header=None)
test = pd.read_csv("../input/data/test/test.tsv",sep='\t',header=None)
process_external_data = pd.read_csv("../input/data/process_external_data/all_csv.tsv",sep='\t',header=None)
process_external_data.iloc[:,1] = process_external_data.iloc[:,1].apply(lambda x: x.strip())
# origin_external_data = pd.read_csv("../input/data/external_data/train.tsv",sep='\t',header=None)

process_vc = process_external_data.iloc[:,8].value_counts()
train_vc = train.iloc[:,8].value_counts()
from transformers import AutoTokenizer
# model_name_from_pretrained = "monologg/koelectra-base-v3-discriminator"
model_name_from_pretrained = "xlm-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name_from_pretrained)


In [242]:
train.head(3)

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭


In [243]:
train['token_len'] = train.iloc[:,1].apply(lambda x: len(tokenizer(x)['input_ids']))
test['token_len'] = test.iloc[:,1].apply(lambda x: len(tokenizer(x)['input_ids']))
process_external_data['token_len'] = process_external_data.iloc[:,1].apply(lambda x: len(tokenizer(x)['input_ids']))

Token indices sequence length is longer than the specified maximum sequence length for this model (675 > 512). Running this sequence through the model will result in indexing errors


## Q3와 Q1 이용 문장길이 필터링

In [244]:
# train_token_Q1 = train['token_len'].quantile(.25)
# train_token_Q3 = train['token_len'].quantile(.75)
# train_IQR = train_token_Q3-train_token_Q1
# test_token_Q1 = test['token_len'].quantile(.25)
# test_token_Q3 = test['token_len'].quantile(.75)
# test_IQR = test_token_Q3-test_token_Q1
# print(train_token_Q1,train_token_Q3,train_IQR)
# print(test_token_Q1,test_token_Q3,test_IQR)

In [245]:
# # train의 Q3 와 Q1 이용
# print(process_external_data.shape)
# condition = (train_token_Q1<process_external_data['token_len'])&(process_external_data['token_len']<train_token_Q3)
# external_data_filtered_q3_q1 = process_external_data.loc[condition,:]
# print(external_data_filtered_q3_q1.shape)
# external_data_filtered_q3_q1.iloc[:,8].value_counts()

## 최대 최소값 이용한 문장길이 필터링

In [ ]:
print(train['token_len'].min(),train['token_len'].max())
print(test['token_len'].min(),test['token_len'].max())
print(process_external_data['token_len'].min(),process_external_data['token_len'].max())
print(process_external_data.shape)
# condition = (test['token_len'].min()<process_external_data['token_len'])&(process_external_data['token_len']<test['token_len'].max())
condition = (train['token_len'].min()<process_external_data['token_len'])&(process_external_data['token_len']<train['token_len'].max())
process_external_data.loc[condition,:].shape

13 341
13 224
5 5615
(229611, 10)


(225578, 10)

In [ ]:
process_external_data.iloc[:,8].value_counts()

인물:출신성분/국적    77850
인물:직업/직함      34615
단체:창립일        27757
인물:제작         16601
단체:제작         15039
단체:구성원        13779
인물:배우자        11205
인물:자녀         10461
인물:기타_친족       9650
인물:부모님         4566
단체:상위_단체       3436
인물:학교          2062
단체:정치/종교성향     1591
인물:소속단체         875
단체:창립자          124
Name: 8, dtype: int64

In [62]:
external_max_min_dict = {}
for label_name in train_vc.index:
  # label_name = '인물:학교'
  token_len_for_label = process_external_data[process_external_data[8]==label_name]['token_len']
  external_max_min_dict[label_name]=[token_len_for_label.max(),token_len_for_label.min()]

train_max_min_dict = {}
for label_name in train_vc.index:
  # label_name = '인물:학교'
  token_len_for_label = train[train[8]==label_name]['token_len']
  train_max_min_dict[label_name]=[token_len_for_label.max(),token_len_for_label.min()]

In [79]:
train_max_min_dict
external_max_min_dict

{'관계_없음': [nan, nan],
 '단체:구성원': [279, 5],
 '인물:소속단체': [91, 13],
 '인물:직업/직함': [247, 5],
 '단체:상위_단체': [129, 12],
 '단체:별칭': [nan, nan],
 '인물:출신성분/국적': [317, 4],
 '인물:동료': [nan, nan],
 '단체:제작': [269, 5],
 '인물:부모님': [258, 7],
 '단체:본사_도시': [nan, nan],
 '단체:본사_국가': [nan, nan],
 '인물:별칭': [nan, nan],
 '인물:배우자': [523, 6],
 '인물:자녀': [317, 5],
 '단체:하위_단체': [nan, nan],
 '단체:창립일': [252, 5],
 '인물:기타_친족': [317, 5],
 '인물:제작': [4768, 5],
 '인물:형제/자매/남매': [nan, nan],
 '단체:창립자': [78, 14],
 '인물:사망_일시': [nan, nan],
 '단체:모회사': [nan, nan],
 '인물:출생_일시': [nan, nan],
 '인물:거주_국가': [nan, nan],
 '인물:거주_도시': [nan, nan],
 '단체:본사_주(도)': [nan, nan],
 '단체:정치/종교성향': [218, 6],
 '단체:해산일': [nan, nan],
 '인물:종교': [nan, nan],
 '인물:거주_주(도)': [nan, nan],
 '인물:용의자': [nan, nan],
 '단체:주주': [nan, nan],
 '단체:구성원_수': [nan, nan],
 '인물:학교': [215, 7],
 '인물:출생_국가': [nan, nan],
 '인물:사망_원인': [nan, nan],
 '인물:나이': [nan, nan],
 '단체:자회사': [nan, nan],
 '인물:출생_도시': [nan, nan],
 '인물:사망_도시': [nan, nan],
 '인물:사망_국가': [nan, nan]}

# https://rfriend.tistory.com/411

# 수지님이 주신 데이터셋 2

In [1]:
# http://boostcamp.stages.ai/competitions/4/discussion/post/174
import pandas as pd
# train = pd.read_csv("../input/data/train/train.tsv",sep='\t',header=None)
# test = pd.read_csv("../input/data/test/test.tsv",sep='\t',header=None)
bornin_city = pd.read_csv("../input/data/process_external_data/19_bornin_city.tsv",sep='\t',header=0)
bornin_country = pd.read_csv("../input/data/process_external_data/26_bornin_country.tsv",sep='\t',header=0)
diedin_city = pd.read_csv("../input/data/process_external_data/37_diedin_city.tsv",sep='\t',header=0)
diedin_country = pd.read_csv("../input/data/process_external_data/40_diedin_country.tsv",sep='\t',header=0)

label_type = pd.read_pickle("../input/data/label_type.pkl")
reverse_label_dict = {v:k for k,v in label_type.items()}

# minor_concat_data = pd.concat([bornin_city,bornin_country,diedin_city,diedin_country],axis=0)
# print(minor_concat_data.shape)
# minor_concat_data['label'] = minor_concat_data['label'].apply(lambda x: reverse_label_dict[x])
# minor_concat_data.loc[:,'sentence'] = minor_concat_data.loc[:,'sentence'].apply(lambda x: x.strip())

bornin_city['label'] = bornin_city['label'].apply(lambda x: reverse_label_dict[x])
bornin_country['label'] = bornin_country['label'].apply(lambda x: reverse_label_dict[x])
diedin_city['label'] = diedin_city['label'].apply(lambda x: reverse_label_dict[x])
diedin_country['label'] = diedin_country['label'].apply(lambda x: reverse_label_dict[x])

bornin_city.loc[:,'sentence'] = bornin_city.loc[:,'sentence'].apply(lambda x: x.strip())
bornin_country.loc[:,'sentence'] = bornin_country.loc[:,'sentence'].apply(lambda x: x.strip())
diedin_city.loc[:,'sentence'] = diedin_city.loc[:,'sentence'].apply(lambda x: x.strip())
diedin_country.loc[:,'sentence'] = diedin_country.loc[:,'sentence'].apply(lambda x: x.strip())

# def condition(x):
#   return ("'" in x or ";" in x or ":" in x or '"' in x or '@' in x or '#' in x or '$' in x or '^' in x or '&' in x or '+' in x or '/' in x or '*' in x or '*' in x or '*' in x)
import re
def check_punc(word):
  reg = re.compile(r'[『』=+#/\?:^$.@*\"※~&%·!\\|\(\)\[\]\<\>`\'《》…]')
  if reg.match(word):
    return True
  else:
    return False

mydata = bornin_city
print(mydata.shape)
# print(sum(mydata['sentence'].apply(lambda x: not condition(x))))



# print(sum(bornin_city['sentence'].apply(lambda x: 1 if "'" in x else 0)))
# print(sum(bornin_city['sentence'].apply(lambda x: 1 if "`" in x else 0)))
# print(sum(bornin_city['sentence'].apply(lambda x: 1 if ";" in x else 0)))
# print(sum(bornin_city['sentence'].apply(lambda x: 1 if '"' in x else 0)))
# print(sum(bornin_city['sentence'].apply(lambda x: True if '※' in x else False)))
# print(sum(bornin_city['sentence'].apply(lambda x: True if '·' in x else False)))
# print(sum(bornin_city['sentence'].apply(lambda x: True if "?" in x else False)))

# ['id','sentence','entity_01','ent1_start','ent1_end','entity_02','ent2_start','ent2_end','label']

(15402, 4)
0
325


In [10]:
def cleanText(data):
  text = re.sub('[『』=+#/\?:^$.@*\"※~&%·!\\|\(\)\{\}\[\]\<\>`\'《》…]','',data)
  return text

data = "!?@#abcdef<>{}[]()"
cleanText(data)

'abcdef'

In [178]:
def num_punc(data,pattern):
  return sum(data[1].apply(lambda x: True if pattern in x else False))
punc_list = ["『","』","=","+","#","/","\\","?",":",";","^","$",".","@","*","\"","※","~","&","%","·","!","|","(",")","{","}","[","]","<",">","`","\'","《","》","…"]

train_num_punc_dict = {punc:num_punc(train,punc) for punc in punc_list}
test_num_punc_dict = {punc:num_punc(test,punc) for punc in punc_list}

tr_no_punc=[punc for punc in punc_list if num_punc(train,punc)==0]
tr_exist_punc=[punc for punc in punc_list if num_punc(train,punc)!=0]

te_no_punc=[punc for punc in punc_list if num_punc(test,punc)==0]
te_exist_punc=[punc for punc in punc_list if num_punc(test,punc)!=0]


# 『』=+#/\?:^$.@*\"※~&%·!\\|\(\)\{\}\[\]\<\>`\'《》…

# for punc in punc_list:
#   print(f"test 중 {punc}의 개수 {num_punc(test,punc)}",end='\n'+'='*20+'\n')

# num_punc(train,"『")
# num_punc(train,"』")
# num_punc(train,"=")
# num_punc(train,"+")
# num_punc(train,"#")
# num_punc(train,"/")
# num_punc(train,"?")
# num_punc(train,":")
# num_punc(train,";")
# num_punc(train,"^")
# num_punc(train,"$")
# num_punc(train,".")
# num_punc(train,"@")
# num_punc(train,"*")
# num_punc(train,'\"')
# num_punc(train,"※")
# num_punc(train,"~")
# num_punc(train,"&")
# num_punc(train,"%")
# num_punc(train,"·")
# num_punc(train,"!")
# num_punc(train,"\\")
# num_punc(train,"|")
# num_punc(train,"(")
# num_punc(train,")")
# num_punc(train,"{")
# num_punc(train,"}")
# num_punc(train,"[")
# num_punc(train,"]")
# num_punc(train,"<")
# num_punc(train,">")
# num_punc(train,"`")
# num_punc(train,"\'")
# num_punc(train,"《")
# num_punc(train,"》")
# num_punc(train,"…")

# train과 test 모두 있는 특수기호들
# / : ; . "

# train에는 있지만 test에는 없는 기호
## 『 』 =  $ * `



In [179]:
# train_num_punc_dict
# test_num_punc_dict

In [180]:
train[8].value_counts()

관계_없음          4441
단체:구성원          815
인물:소속단체         679
인물:직업/직함        553
단체:상위_단체        335
단체:별칭           231
인물:출신성분/국적      209
인물:동료           186
단체:제작           165
인물:부모님          164
단체:본사_도시        136
단체:본사_국가        115
인물:별칭           103
인물:배우자           92
인물:자녀            84
단체:하위_단체         80
단체:창립일           67
인물:기타_친족         58
인물:제작            56
인물:형제/자매/남매      52
인물:사망_일시         45
단체:창립자           45
단체:모회사           44
인물:출생_일시         36
인물:거주_국가         27
인물:거주_도시         26
단체:본사_주(도)       23
단체:정치/종교성향       18
인물:종교            15
단체:해산일           15
인물:거주_주(도)       12
인물:용의자           11
단체:주주            11
인물:학교             9
단체:구성원_수          9
인물:출생_국가          8
인물:사망_원인          7
단체:자회사            5
인물:나이             5
인물:출생_도시          4
인물:사망_도시          3
인물:사망_국가          1
Name: 8, dtype: int64

In [181]:
for k,v in train_num_punc_dict.items():
  print(f"{k} : {v}개")

『 : 8개
』 : 8개
= : 2개
+ : 0개
# : 0개
/ : 55개
\ : 0개
? : 0개
: : 105개
; : 5개
^ : 0개
$ : 5개
. : 9000개
@ : 0개
* : 4개
" : 655개
※ : 0개
~ : 368개
& : 62개
% : 206개
· : 490개
! : 28개
| : 4개
( : 2683개
) : 2688개
{ : 0개
} : 0개
[ : 21개
] : 21개
< : 0개
> : 0개
` : 7개
' : 667개
《 : 190개
》 : 189개
… : 0개


In [89]:
print(f"train과 test 모두 없는 기호들 : {list(set(tr_no_punc)&set(te_no_punc))}")
print(f"train에만 있는 기호들 : {list(set(tr_exist_punc)-set(te_exist_punc))}")
print(f"test에만 있는 기호들 : {list(set(te_exist_punc)-set(tr_exist_punc))}")
print(f"train과 test 모두 있는 기호들 : {list(set(tr_exist_punc)&set(te_exist_punc))}")

print(f"외부데이터 추가시에 포함되어선 안될 특수기호들 : {list(set(punc_list)-set(te_exist_punc))}")

train과 test 모두 없는 기호들 : ['>', '\\', '+', '※', '…', '^', '@', '{', '?', '}', '<', '#']
train에만 있는 기호들 : ['*', '=', '『', '`', '$', '』']
test에만 있는 기호들 : []
train과 test 모두 있는 기호들 : ['|', '%', '》', '.', '"', '·', '《', '!', '(', '/', ')', '[', ';', ']', "'", '&', ':', '~']
외부데이터 추가시에 포함되어선 안될 특수기호들 : ['?', '>', '\\', '+', '*', '※', '^', '…', '=', '『', '}', '`', '@', '{', '$', '』', '<', '#']


In [83]:
# entitiy1
# train.iloc[:,2]

# entity2
# train.iloc[:,5]

# Train에 있는 특수기호들 처리

## 1. * 은 (4개) 
replace('*','')로 단순 제거해주면 됨

In [93]:
train_num_punc_dict['*']

4

In [92]:
# train에만 있는 기호들
# * 는 replace('*','')로 제거하자.
train[1].loc[train[1].apply(lambda x: True if '*' in x else False)].values

array(["지난 3일 아이즈원은 공식 SNS 채널에 첫 번째 정규앨범 '블룸아이즈(BLOOM*IZ)' 수록곡 메들리와 멤버별 모습을 공개했다.",
       '인천시(시장 박남춘)는 8월 19일(월)부터 8월 23일(금)까지 5일간 인천광역시 연수구 송도컨벤시아(프리미어볼룸)에서 개도국 고위급 인사, 국가지정기구(NDA*), 인증기구(AE**), 시민사회기관(CSO) 관계자 및 GCF 이사 등 800여명이 참석하는 녹색기후기금(GCF) 글로벌 프로그래밍 콘퍼런스가 개최한다고 밝혔다.',
       '‘jp68****’은 아베 신조 일본 총리를 향해 “변태도 수출 규제 품목에 넣어라”라고 촉구했다.',
       '2012년 10월 18일에는 B*Witched가 ITV2의 텔레비전 리얼리티 다큐멘터리 시리즈 《더 빅 리유니언》(The Big Reunion)에 출연하기 위하여 당대의 다른 5개의 팝 음악 그룹인 허니즈(Honeyz), 911, 리버티 X(Liberty X), 5ive, 아토믹 키튼(Atomic Kitten)와 함께 재결합할 것이라는 보도가 전해졌다.'],
      dtype=object)

## 2. = (데이터 2개)

replace('本方=일본','일본')으로 대체

replace('시마즈=아리마','시마즈-아리마')


In [49]:
train_num_punc_dict['=']

2

In [53]:
# train에만 있는 기호들
# = 는 replace('本方=일본','일본')으로 대체
# = 는 replace('시마즈=아리마','시마즈-아리마')
train[1].loc[train[1].apply(lambda x: True if '=' in x else False)].values


array(['내무성은 5개월 간의 조사 결과 이 문제는 이미 1699년에 끝난 문제로 울릉도와 이 섬은 조선 영토이며 ‘본방(本方=일본)과는 관계가 없다’고 결론짓고 일본 지도와 지적 조사에서 제외하기로 결정했다고 한국 학자들은 주장한다.',
       '1584년 일어난 오키타나와테 전투에서 당시 류조지 가문의 당주 류조지 다카노부가 시마즈=아리마 연합군에 패사하였다.'],
      dtype=object)

## 3. 『랑  』(쌍으로 8개)
『 는 replace('『','《') 이중꺾쇠로 바꾸자

』 는 replace('』','》') 이중꺾쇠로 바꾸자

In [133]:
print(train_num_punc_dict['『'])
print(train_num_punc_dict['《'])
print(test_num_punc_dict['《'])


8
190
26


In [99]:
# train에만 있는 기호들
# 『 는 replace('『','[') 제거하자.
# 』 는 replace('』','[') 제거하자.

train.iloc[:,[1,8]].loc[train[1].apply(lambda x: True if '『' in x else False)].values

array([['1992년부터 만화책을 원작으로 『짱구는 못말려』이라는 동명의 이름으로 신에이 동화에서 TV시리즈 애니메이션을 제작하여 테레비 아사히를 비롯한 아사히 방송(긴키 광역권), 나고야 TV(주쿄 광역권), 시즈오카 아사히 TV, 규슈 아사히 방송, 히로시마 홈 TV 등 24개 지역 민방 네트워크를 통해 방영되고 있다.',
        '단체:제작'],
       ['일본사회당을 서구의 사회민주주의와는 다른 사회민주주의로 받아들이고 그 차이를 긍정적으로 인식하는 분석은 시미즈 신조의 『일본의 사회민주주의』(이와나미 신서, 1961년)에서 비롯되었다.',
        '단체:정치/종교성향'],
       ['그 후 미쓰쿠니는 아버지 요리후사의 죽음으로 인해 가업을 상속하였고, 공무 수행으로 인해 바빠지는 바람에 편수 사업으로부터는 멀어지게 되었지만, 한편으로 막부에서는 간분 2년(1662년)에 하야시 가호(林鵞峰)에게 명해 편년체의 사서 『본조통감(本朝通鑑)』의 편찬을 개시하였고, 미쓰쿠니는 하야시 가보를 번저에 초대하여 면담하며, 편찬 방침이나 정통성 문제에 대해 질의하였다.',
        '관계_없음'],
       ['SK하이닉스가 협력사 대상 지식공유 플랫폼인 ‘반도체 아카데미’를 운영하며 축적된 반도체 전문지식과 경험을 공유하기 위하여 『패키지와 테스트(원제 : 반도체의 부가가치를 올리는 패키지와 테스트)』 책을 10일 펴냈다.',
        '관계_없음'],
       ['사단장은 북한군이 그대로 남진할 경우 서부전선이 위급하게 될 것임을 직감하고, 제7연대장 임부택 중령에게 『장호원을 즉각 탈취하라.』고 명령하였다.',
        '인물:동료'],
       ['일본사회당을 서구의 사회민주주의와는 다른 사회민주주의로 받아들이고 그 차이를 긍정적으로 인식하는 분석은 시미즈 신조의 『일본의 사회민주주의』(이와나미 신서, 1961년)에서 비롯되었다.',
        '관계_없음'],
       ['반면, 『매송론』(梅松

## 5. $ (5개)

R$ 는 replace('R$','금액') 으로 대체하자.

AU$ 는 replace('AU$','금액') 으로 대체하자.

$ 는 replace('$','금액') 으로 대체하자.


In [109]:
# train에만 있는 기호들
# R$ 는 replace('R$','금액') 제거하자.
# AU$ 는 replace('AU$','금액') 으로 대체하자.
# $ 는 replace('$','금액') 으로 대체하자.

train.iloc[:,[1,2,5,8]].loc[train[1].apply(lambda x: True if '$' in x else False)].values

array([['이지연 신영증권 연구원은 “SK이노베이션의 올해 1분기 영업이익은 -1180억원으로 적자가 불가피할 전망”이라며 “코로나19의 영향으로 운송수요가 급감하였으며, 평년보다 따뜻한 날씨로 정제마진은 지난해 4분기 $2.1에서 올해 1분기 $0.5로 하락한 상황”이라고 말했다.',
        'SK이노베이션', '연구원', '관계_없음'],
       ['2011년 7월 비도시치는 AU$320,000의 이적료로 A리그 애들레이드 유나이티드 FC와 3년 계약을 맺으며, 오스트레일리아로 복귀했다.',
        '애들레이드 유나이티드 FC', 'A리그', '단체:상위_단체'],
       ['필리포 인차기 신임 감독의 계획에서 제외되었음에도 불구하고, 이탈리아 구단 측이 호비뉴의 주급을 지불하고 "물고기 군단"측이 월급으로 R$400,000을 부담하고, 밀란이 R$400,000을 책임질 것으로 발표되었다.',
        '밀란', '필리포 인차기', '관계_없음'],
       ['그는 Ludacris(루다크리스)와 폰티액 간에 $1,200만 캠페인 거래를 중개하여 자신의 마케팅 비즈니스를 시작하였고, Pontiac의 광고에는 노래가 나오는 한편, Ludacris의 1시간에 2마일" 뮤직 비디오에는 Pontiac이 나오게 된다.',
        '루다크리스', 'Ludacris', '단체:별칭'],
       ['위원회는 정부의 공식적인 사과와 각 생존자에게 $20,000의 보상을 지불하는 등의 법적인 구제를 권장하였다.',
        '정부', '생존자', '관계_없음']], dtype=object)

## 6. ` (7개)

replace("괴물 신인`","괴물 신인")

나머진 replace("`","'") 로 대체하자

In [111]:
# train에만 있는 기호들
# replace("괴물 신인`","괴물 신인")
# 웬만하면 replace("`","'") 로 대체하자
# AU$ 는 replace('AU$','금액') 으로 대체하자.
# $ 는 replace('$','금액') 으로 대체하자.

train.iloc[:,[1,2,5,8]].loc[train[1].apply(lambda x: True if '`' in x else False)].values

array([['‘2020 대한민국 수공예공모대전’에서 강진군 청자업체인 토우 김유성 대표가 `청자 연리문 금채 다기`를 출품해 금상을 수상했다.',
        '김유성', '대한민국', '인물:출신성분/국적'],
       ["괴물 신인` ITZY(있지)가 JYP엔터테인먼트(이하 JYP) 박진영 대표가 작사, 작곡한 `ICY(아이씨)'로 컴백한다.",
        '박진영', 'JYP', '인물:소속단체'],
       ['2001년 11월 30일 민주노동당 황광우 등이 민주노동당 기관지와 인터넷 홈페이지 등에 `사회당 동지들에게 드리는 7가지 질문\'이라는 글 등을 싣고 "조선노동당(조선로동당)은 사회당의 적이냐"고 묻자 12월 11일 한국사회당은 모든 종류의 테러나 전쟁에 반대하며 "남한의 노동계급을 이끌고 북한에 쳐들어가 조선노동당을 물리치는 일이 국가간 전쟁의 범주에 속한다"고 반박하는 한편, "조선노동당의 사회관이 관철되는 통일에는 단연코 반대한다" 고 밝힘으로서 시작되었고 사회당은 \'친북\'과 구별하기 위해 \'종북\'이라는 단어를 사용했다는 주장이있다.',
        '사회당', '2001년', '관계_없음'],
       ['2001년 11월 30일 민주노동당 황광우 등이 민주노동당 기관지와 인터넷 홈페이지 등에 `사회당 동지들에게 드리는 7가지 질문\'이라는 글 등을 싣고 "조선노동당(조선로동당)은 사회당의 적이냐"고 묻자 12월 11일 한국사회당은 모든 종류의 테러나 전쟁에 반대하며 "남한의 노동계급을 이끌고 북한에 쳐들어가 조선노동당을 물리치는 일이 국가간 전쟁의 범주에 속한다"고 반박하는 한편, "조선노동당의 사회관이 관철되는 통일에는 단연코 반대한다" 고 밝힘으로서 시작되었고 사회당은 \'친북\'과 구별하기 위해 \'종북\'이라는 단어를 사용했다는 주장이있다.',
        '사회당', '2001년', '관계_없음'],
       ['88년 노무현, 이해찬 국회의원과 함께 국회 노동위원회에서 활동하며 `노동위 3총

In [ ]:
minor_concat_data['sentence'].iloc[4].replace("'","")

NameError: name 'minor_concat_data' is not defined

In [27]:
bornin_city.apply(lambda x: x['sentence'].index(x['entity_01']),axis=1).argmin()

46

# 외부데이터에서 특수기호 포함하는 행 제거

In [ ]:
# ['?', '>', '\\', '+', '*', '※', '^', '…', '=', '『', '}', '`', '@', '{',
# '$', '』', '<', '#']
import re
def exist_punc(word):
    reg = re.compile('[?><+*※^…=『』`@$#\{\}\\\]')
    if reg.search(word):
        # print("특수문자 포함")
        # print(reg.search(word))
        return True
    else:
        # print("특수문자 없음")
        # print(reg.search(word))
        return False

# bornin_city['sentence'].apply(lambda x: exist_punc(x))

Hello world\
특수문자 포함
<re.Match object; span=(11, 12), match='\\'>


True

In [20]:
# print(bornin_city.shape)
# print(bornin_country.shape)
# print(diedin_city.shape)
# print(diedin_country.shape)


In [8]:
train_vc

관계_없음          4441
단체:구성원          815
인물:소속단체         679
인물:직업/직함        553
단체:상위_단체        335
단체:별칭           231
인물:출신성분/국적      209
인물:동료           186
단체:제작           165
인물:부모님          164
단체:본사_도시        136
단체:본사_국가        115
인물:별칭           103
인물:배우자           92
인물:자녀            84
단체:하위_단체         80
단체:창립일           67
인물:기타_친족         58
인물:제작            56
인물:형제/자매/남매      52
단체:창립자           45
인물:사망_일시         45
단체:모회사           44
인물:출생_일시         36
인물:거주_국가         27
인물:거주_도시         26
단체:본사_주(도)       23
단체:정치/종교성향       18
단체:해산일           15
인물:종교            15
인물:거주_주(도)       12
단체:주주            11
인물:용의자           11
인물:학교             9
단체:구성원_수          9
인물:출생_국가          8
인물:사망_원인          7
단체:자회사            5
인물:나이             5
인물:출생_도시          4
인물:사망_도시          3
인물:사망_국가          1
Name: 8, dtype: int64

In [ ]:

train_vc = train.iloc[:,8].value_counts()
from transformers import AutoTokenizer
# model_name_from_pretrained = "monologg/koelectra-base-v3-discriminator"
model_name_from_pretrained = "xlm-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name_from_pretrained)